## Status code

200 -> A requisição deu certo

300 -> Redirecionada

400 -> Não conseguiu fazer a requisição

500 -> Deu um erro no sistema/url

In [ ]:
import requests
import pandas as pd

# URLs da API
url_produto = 'https://fakestoreapi.com/products'
url_usuarios = 'https://fakestoreapi.com/users'
url_carrinhos = 'https://fakestoreapi.com/carts'

# Requisição para os produtos
res_produtos = requests.get(url_produto)
if res_produtos.status_code == 200:
    df_produtos = pd.DataFrame(res_produtos.json())
    print("Tabela de Produtos")
    print(df_produtos.head(5))
else:
    print(f"Erro ao acessar produtos: {res_produtos.status_code}")

# Requisição para os usuários
res_usuarios = requests.get(url_usuarios)
if res_usuarios.status_code == 200:
    df_usuarios = pd.DataFrame(res_usuarios.json())
    print("Tabela de Usuários")
    print(df_usuarios.head(5))
else:
    print(f"Erro ao acessar usuários: {res_usuarios.status_code}")

# Requisição para os carrinhos
res_carrinhos = requests.get(url_carrinhos)
if res_carrinhos.status_code == 200:
    df_carrinhos = pd.DataFrame(res_carrinhos.json())
    print("Tabela de Carrinhos")
    print(df_carrinhos.head(5))
else:
    print(f"Erro ao acessar carrinhos: {res_carrinhos.status_code}")

Tabela de Produtos
   id                                              title   price  \
0   1  Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...  109.95   
1   2             Mens Casual Premium Slim Fit T-Shirts    22.30   
2   3                                 Mens Cotton Jacket   55.99   
3   4                               Mens Casual Slim Fit   15.99   
4   5  John Hardy Women's Legends Naga Gold & Silver ...  695.00   

                                         description        category  \
0  Your perfect pack for everyday use and walks i...  men's clothing   
1  Slim-fitting style, contrast raglan long sleev...  men's clothing   
2  great outerwear jackets for Spring/Autumn/Wint...  men's clothing   
3  The color could be slightly different between ...  men's clothing   
4  From our Legends Collection, the Naga was insp...        jewelery   

                                               image  \
0  https://fakestoreapi.com/img/81fPKd-2AYL._AC_S...   
1  https://fakestoreapi.com